In [29]:
from PyPDF2 import PdfReader
from langchain.document_loaders import OnlinePDFLoader
from langchain.embeddings.ollama import OllamaEmbeddings
from langchain.indexes.vectorstore import VectorstoreIndexCreator
from langchain_community.llms import Ollama
from langchain.chains.question_answering import load_qa_chain
from langchain.text_splitter import CharacterTextSplitter
# from langchain_pinecone import PineconeVectorStore
from langchain_community.vectorstores import FAISS
from typing_extensions import Concatenate

In [2]:
reader = PdfReader("gpt-4.pdf")

In [3]:
raw_text = ""
for page in reader.pages:
    raw_text += page.extract_text()

In [4]:
raw_text[:30]

'GPT-4 Technical Report\nOpenAI\x03'

In [5]:
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=1000,
    chunk_overlap=300,
    length_function=len
)

In [6]:
text = text_splitter.split_text(raw_text)

In [7]:
text[:2]

['GPT-4 Technical Report\nOpenAI\x03\nAbstract\nWe report the development of GPT-4, a large-scale, multimodal model which can\naccept image and text inputs and produce text outputs. While less capable than\nhumans in many real-world scenarios, GPT-4 exhibits human-level performance\non various professional and academic benchmarks, including passing a simulated\nbar exam with a score around the top 10% of test takers. GPT-4 is a Transformer-\nbased model pre-trained to predict the next token in a document. The post-training\nalignment process results in improved performance on measures of factuality and\nadherence to desired behavior. A core component of this project was developing\ninfrastructure and optimization methods that behave predictably across a wide\nrange of scales. This allowed us to accurately predict some aspects of GPT-4’s\nperformance based on models trained with no more than 1/1,000th the compute of\nGPT-4.\n1 Introduction',
 'infrastructure and optimization methods tha

In [ ]:
embeddings = OllamaEmbeddings()
search = FAISS.from_texts(text, embeddings)

In [ ]:
llm = Ollama(model="gemma:2b")
chain = load_qa_chain(llm=llm, chain_type="refine")

In [ ]:
query = "why is this report important?"
pdf = search.similarity_search(query) 
chain.run(pdf, query)

In [ ]:
loader = OnlinePDFLoader("abc.pdf")

In [ ]:
data = loader.load()
embeddings = OllamaEmbeddings()
index = VectorstoreIndexCreator().from_loaders([loader], embeddings)


In [ ]:
query =  "why is this report important?"
index.query(query)